# input data

In [ ]:
#Import statements
import random
import pandas as pd
pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV #built in class to provide tuning
from sklearn.model_selection import RepeatedKFold
from numpy import arange
from sklearn.neural_network import MLPRegressor
from sklearn import preprocessing
random.seed(1) #set a seed for reproducable result

In [ ]:
def combine_embedding_with_table(embedding_name,modelling_variable):

    """
    Combine  embedding  with a basetable and performs some data preprocessing.

    Args:
    - embedding_name (str): the name of the embedding ('glove', 'elmo', 'berttwitter', 'sentencebert', 'universal').
    - modelling_variable (str): the name of  ('SWB', 'Positive_affect', 'Negative_affect').

    Returns:
    - DataFrame: The combined embedding and basetable

    """

    basetable = pd.read_csv("./data/ModelData.csv")
    try:
        # Construct file path of embedding
        filename = f"./data/{embedding_name}.csv"

        # Read the CSV file into a DataFrame
        emb_table = pd.read_csv(filename)

        # Concatenate the basetable and the embedding
        basetable = pd.concat([basetable, emb_table], axis=1)

        # List of column names to delet
       # List of column names to delet
        columns_to_delete = ['created_at', 'full_text',
                     'Attentive','Alert','Determined','Inspired','Active',
                     'Hostile','Ashamed','Upset','Afraid','Nervous',
                     'OriginalText','clean_text','neu' ]
        basetable = basetable.drop(columns=columns_to_delete)

        # List of column names to delete based on modelling_variable
        if modelling_variable == 'SWB':
            columns_to_delete = ['Positive_affect', 'Negative_affect']
        elif modelling_variable == 'Positive_affect':
            columns_to_delete = ['SWB', 'Negative_affect']
        elif modelling_variable == 'Negative_affect':
            columns_to_delete = ['SWB', 'Positive_affect']
        else:
            columns_to_delete = []
        basetable = basetable.drop(columns=columns_to_delete)



        print(f"Combined {embedding_name} with basetable.")
        print(f"Dropped columns: {columns_to_delete}")
        print("=" * 40)
        print(f"This code is used to build a model of {modelling_variable} using {embedding_name}")

        return basetable

    except FileNotFoundError:
        print(f"File '{embedding_name}' not found.")
        return basetable


# SWB

In [ ]:
##here input two , one is embedding name, another is the dependent variable to build a model
#('glove', '1024elmo', 'berttwitter', 'sentencebert', 'universal')
#('SWB', 'Positive_affect', 'Negative_affect')
basetable=combine_embedding_with_table('openai','SWB')

In [ ]:
basetable.shape


In [ ]:
basetable

In [ ]:
#now create date
covid_start = pd.to_datetime('2020-01-30', utc = True)
basetable['created_at'] = covid_start + pd.to_timedelta(basetable['days_since_COVID19'], unit='D')
basetable['created_at'] = basetable['created_at'].dt.date


In [ ]:
basetable = basetable.sort_values(by='created_at')

In [ ]:
basetable

In [ ]:
X = basetable.drop(['SWB','created_at','neg','pos',	'compound',	'TextBlob_score','days_since_COVID19'], axis = 1)
y = basetable['SWB']

X["is_quote_status"] = X["is_quote_status"].astype(int)
X["is_reply"] = X["is_reply"].astype(int)
X["possibly_sensitive"] = X["possibly_sensitive"].astype(int)

In [ ]:
X

In [ ]:
# Define the ratio for the split
split_ratio = 0.7  # 70% for training, 30% for testing

# Calculate the index to split the data
split_index = int(len(X) * split_ratio)

# Split the data into training and testing sets
X_train = X[:split_index]
y_train = y[:split_index]
X_test = X[split_index:]
y_test = y[split_index:]

In [ ]:
X_train

In [ ]:
##here we use NN+openai

In [ ]:
model =  MLPRegressor(random_state=1)

# defining parameter range
param_grid = {'hidden_layer_sizes': [2,4,6,8,10,12,14,16,18,20],
              'batch_size':[32],
             'alpha': [10**(-4), 10**(-3.5), 10**(-3), 10**(-2.5), 10**(-2), 10**(-1.5), 10**(-1), 10**(-0.5), 0]}



#sclaing part of features
scale_columns=['retweet_count','favorite_count',
                   'hour','url_count','mentions_count','questionmark_count',
                   'exclamationmark_count','points_count','uppercase_ratio',
                   'hashtag_count']

#select the columns to scale
X_train_scale = X_train[scale_columns]
X_test_scale = X_test[scale_columns]

X_train_unscale = X_train.drop(columns=scale_columns)
X_test_unscale = X_test.drop(columns=scale_columns)

scaler = StandardScaler().fit(X_train_scale)
X_train_transformed = pd.DataFrame(scaler.transform(X_train_scale), columns=scale_columns)
X_test_transformed = pd.DataFrame(scaler.transform(X_test_scale), columns=scale_columns)


# Reset the indexes of both DataFrames
X_train_unscale.reset_index(drop=True, inplace=True)
X_train_transformed.reset_index(drop=True, inplace=True)
X_test_unscale.reset_index(drop=True, inplace=True)
X_test_transformed.reset_index(drop=True, inplace=True)
# Concatenate the DataFrames together
X_train = pd.concat([X_train_transformed,X_train_unscale,], axis=1)
X_test = pd.concat([X_test_transformed,X_test_unscale], axis=1)



In [ ]:

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=1, verbose=1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_
print('Config: %s' % grid_search.best_params_)

final_model = best_model.fit(X_train, y_train)
y_pred = final_model.predict(X_test)
y_train_pred=final_model.predict(X_train)

y_pred = np.clip(y_pred, -1, 1)
y_train_pred== np.clip(y_train_pred, -1, 1)

model_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
model_r2 = r2_score(y_test, y_pred)
print('r2', model_r2)
print('RMSE', model_rmse)



In [ ]:
predict_SWB= np.concatenate((y_train_pred, y_pred), axis=0)

In [ ]:
basetable['predict_SWB']=predict_SWB

In [ ]:
basetable['created_at'] = pd.to_datetime(basetable['created_at'])

In [ ]:
# Group by month and calculate the mean of 'Negative_affect', 'predict_neg', and 'neg'
mean_SWB = basetable.resample('M', on='created_at')['SWB'].mean()
mean_predict_SWB = basetable.resample('M', on='created_at')['predict_SWB'].mean()
mean_vader_compound = basetable.resample('M', on='created_at')['compound'].mean()
mean_textblob_score = basetable.resample('M', on='created_at')['TextBlob_score'].mean()



In [ ]:
mean_predict_SWB

In [ ]:
end_date = pd.to_datetime('2020-02-29')
first_months_data = basetable[basetable['created_at'] <= end_date]
# Calculate the mean for the desired columns
first_months_mean_SWB = first_months_data['SWB'].mean()
first_months_mean_predict_SWB = first_months_data['predict_SWB'].mean()
first_months_mean_vader_compound = first_months_data['compound'].mean()
first_months_mean_textblob_score = first_months_data['TextBlob_score'].mean()

In [ ]:
#scaling
mean_SWB=mean_SWB/np.abs(first_months_mean_SWB)
mean_predict_SWB=mean_predict_SWB/np.abs(first_months_mean_predict_SWB)
mean_vader_compound=mean_vader_compound/np.abs(first_months_mean_vader_compound)
mean_textblob_score=mean_textblob_score/np.abs(first_months_mean_textblob_score)


mean_SWB=mean_SWB[1:]
mean_predict_SWB=mean_predict_SWB[1:]
mean_vader_compound=mean_vader_compound[1:]
mean_textblob_score=mean_textblob_score[1:]

mean_SWB[0]=1
mean_predict_SWB[0]=1
mean_vader_compound[0]=1
mean_textblob_score[0]=1

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('default')  # Resetting to default style

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 6))
# Plot all three variables on the same plot
mean_SWB.plot(label='Total SWB', linewidth=2.5,legend=False)
mean_predict_SWB.plot(label='Predicted total SWB',linewidth=2.5, legend=True)
mean_vader_compound.plot(label='VADER compound', linewidth=2.5,legend=True)
mean_textblob_score.plot(label='TextBlob', linewidth=2.5,legend=True)
ax.tick_params(axis='both',labelsize=16)


# Add a vertical line at January 2021
plt.axvline(pd.to_datetime('2021-01-01'), color='black', linestyle='--')
ax.annotate('Training period', xy=(0.25, 0.4), fontsize=16, color='blue',xycoords='figure fraction')
ax.annotate('Deployment period', xy=(0.7, 0.4), fontsize=16, color='blue',xycoords='figure fraction')

# Add a grid
ax.grid(True, linestyle='--', alpha=0.7)

# Customize labels and title
ax.set_xlabel('')
ax.tick_params(labelsize=16)

plt.ylabel('Values',fontsize=16)

#plt.title('Mean Negative Affect, Mean Predicted Negative Affect, and Mean VADER Negative Over Months', fontsize=12)

# Add a legend with a border
#ax.legend(loc='upper left', frameon=False, framealpha=0.5)

# Rotate x-axis labels for readability
plt.xticks(rotation=45)
plt.legend(loc='upper left', bbox_to_anchor=(0.2, 1), fontsize='large')

# Tighten layout for better spacing
plt.tight_layout()
plt.savefig('TrackSWBScale.png')


# Show the plot
plt.show()